#### Juan Pablo Fonseca Correa  138263
### Ejercicio de validación cruzada para determinar la mejor lambda

In [1]:
# importar librerías y leer la base de datos
import pandas as pd
import numpy as np
from scipy import sparse
from sys import maxint
from math import pow
from math import fabs
from __future__ import division
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import preprocessing
import matplotlib.pyplot as plt
from random import uniform
df = pd.read_csv('regLinPoli2.csv') # leo la bd 

In [2]:
# separar en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[df.columns[-1]], train_size=0.75)
p = len(X_train.columns)

c:\users\juanpa~1\desktop\env1\python~1\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
def Vap(x, coeff):
    vap = 0
    for i in range(0,p+1):
        vap += coeff[i]*x[i]
    return vap

In [23]:
# error siempre calculado sobre el conjunto de APRENDIZAJE
def errorK(coeff,j):
    y = coeff[0]
    for i in range(0,p):
        y += coeff[i+1]*cjAprend_sc[j][:,i]
    errores = [pow(y[i]-yAprend[j].iloc[i],2) for i in range(0,len(yAprend[j].values))]
    error = sum(errores)
    return error

In [24]:
k = 7 # k-fold cross validation; la k se puede modificar en esta línea
X_trainK = []
Y_trainK = []
noTuplas = X_train.shape[0]
for i in range(0,k):
    X_trainK.append(X_train.iloc[int(i*noTuplas/k):int((i+1)*noTuplas/k)-1,:])
    Y_trainK.append(Y_train.iloc[int(i*noTuplas/k):int((i+1)*noTuplas/k)-1])
cjAprend = [] # conjuntos de aprendizaje
cjValid = [] # conjuntos de validación
cjAprend_sc = [] # conjuntos de aprendizaje escalados
cjValid_sc = [] # conjuntos de validación escalados
yAprend = [] # y asociadas con cjAprend
yValid = [] # y asociadas con cjValid
tempX = [] # guardas los dataframes previo a mezclarlos/concatenarlos
tempY = [] # guardas los series previo a mezclarlos
for i in range(0,k):
    for j in range(0,k):
        if j == i:
            cjValid.append(X_trainK[j])
            yValid.append(Y_trainK[j])
        else:
            tempX.append(X_trainK[j])
            tempY.append(Y_trainK[j])
    cjAprend.append(pd.concat(tempX))
    yAprend.append(pd.concat(tempY))
    tempX = []
    tempY = []
    
    scaler = preprocessing.StandardScaler().fit(cjAprend[i])
    cjAprend_sc.append(scaler.transform(cjAprend[i]))
    cjValid_sc.append(scaler.transform(cjValid[i]))    

In [25]:
lambdas = np.arange(0,0.1,0.001) # lambdas que se van a probar
errores = []
for i in range(len(lambdas)):
    err = []
    for j in range(0,k):
        w = [uniform(-5,5) for q in range(0,p+1)]
        eta = 0.005 # fija
        for a in range (0,len(cjAprend_sc[j])):
            xa0 = [1]
            xademas=[cjAprend_sc[j][a][b] for b in range(0,p)]
            xa = xa0 + xademas
            ya = yAprend[j].iloc[a]
            error_actual = ya - Vap(xa,w)
            # actualizas UNA vez para el dato i
            for b in range(len(w)):
                w[b] = w[b] + eta * error_actual * xa[b] - lambdas[i] * w[b]
        # error
        err.append(errorK(w,j))
    print "Para lambda = {0}, el error es: {1}".format(lambdas[i],sum(err))
    errores.append(sum(err))

Para lambda = 0.0, el error es: 1.37916278891e+44
Para lambda = 0.001, el error es: 1.4204113683e+44
Para lambda = 0.002, el error es: 1.50862489081e+44
Para lambda = 0.003, el error es: 1.61066382079e+44
Para lambda = 0.004, el error es: 1.71209072729e+44
Para lambda = 0.005, el error es: 1.80701733662e+44
Para lambda = 0.006, el error es: 1.89347104447e+44
Para lambda = 0.007, el error es: 1.97124051374e+44
Para lambda = 0.008, el error es: 2.04086276444e+44
Para lambda = 0.009, el error es: 2.10314687027e+44
Para lambda = 0.01, el error es: 2.15895386658e+44
Para lambda = 0.011, el error es: 2.20910005817e+44
Para lambda = 0.012, el error es: 2.25431968962e+44
Para lambda = 0.013, el error es: 2.29525569445e+44
Para lambda = 0.014, el error es: 2.33246313599e+44
Para lambda = 0.015, el error es: 2.3664177878e+44
Para lambda = 0.016, el error es: 2.39752620659e+44
Para lambda = 0.017, el error es: 2.42613560783e+44
Para lambda = 0.018, el error es: 2.45254283171e+44
Para lambda = 0.0

In [26]:
indiceLambda = errores.index(min(errores))
lam = lambdas[indiceLambda]
print "La mejor lambda es {0}".format(lam)

La mejor lambda es 0.0


#### Observamos que la mejor lambda en este caso es 0. Como en el primer ejercicio de regularización, los errores son menores con esta lambda que con cualquier otra. Sin embargo, existen bases de datos en las que la mejor lambda sea mayor a cero.
#### Otro punto a considerar es que la eta = 0.005 y estaba fija. Quizá con otra eta, la lambda óptima sería distinta.